## Setup

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.colors
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
from tqdm import tqdm_notebook
from keras import regularizers

from sklearn.preprocessing import OneHotEncoder
from sklearn.datasets import make_blobs




#I have used this (#) to explain!

Using TensorFlow backend.


In [0]:
!wget https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTrain%2B.txt
!wget https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTest%2B.txt
!ls

--2019-10-13 03:17:38--  https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTrain%2B.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19109424 (18M) [text/plain]
Saving to: ‘KDDTrain+.txt’

KDDTrain+.txt       100%[===================>]  18.22M   102MB/s    in 0.2s    

2019-10-13 03:17:40 (102 MB/s) - ‘KDDTrain+.txt’ saved [19109424/19109424]

--2019-10-13 03:17:43--  https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTest%2B.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3441513 (3.3M) [text/plain]
Saving to: ‘

In [0]:
# here, you have taken the input datasets

df_train = pd.read_csv('KDDTrain+.txt', header=None, index_col=None)
df_test = pd.read_csv('KDDTest+.txt', header=None, index_col=None)
df_train.head()
a=pd.DataFrame.read_csv("https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTrain%2B.txt")

AttributeError: ignored

In [0]:
import pandas_profiling
pandas_profiling.ProfilingReport(data)

SyntaxError: ignored

In [0]:

df_train.drop(42, axis=1, inplace=True)
df_test.drop(42, axis=1, inplace=True)

In [0]:
df_train.loc[df_train[41]!='normal', 41] = 1
df_test.loc[df_test[41]!='normal', 41] = 1
df_train.loc[df_train[41]=='normal', 41] = 0
df_test.loc[df_test[41]=='normal', 41] = 0
df_train.groupby(41).count()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
41,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,67343,67343,67343,67343,67343,67343,67343,67343,67343,67343,67343,67343,67343,67343,67343,67343,67343,67343,67343,67343,67343,67343,67343,67343,67343,67343,67343,67343,67343,67343,67343,67343,67343,67343,67343,67343,67343,67343,67343,67343,67343
1,58630,58630,58630,58630,58630,58630,58630,58630,58630,58630,58630,58630,58630,58630,58630,58630,58630,58630,58630,58630,58630,58630,58630,58630,58630,58630,58630,58630,58630,58630,58630,58630,58630,58630,58630,58630,58630,58630,58630,58630,58630


In [0]:
# here you are splitting the data into X and Y matrices

X_train = df_train.drop(41, axis=1)
y_train = df_train.loc[:,[41]]
X_test = df_test.drop(41, axis=1)
y_test = df_test.loc[:,[41]]
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(125973, 41) (125973, 1) (22544, 41) (22544, 1)


In [0]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

#from __future__ import print_function
import os
import time
import json
import numpy as np
import pandas as pd
import sys
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

In [0]:
le = preprocessing.LabelEncoder()
enc = OneHotEncoder()

In [0]:
# one hot encoding to convert string inputs to integer. Sigmoid function NEEDS REAL NUMBER INPUT. Notice the change to column 1, 2 and 3.
# Read this for reference : https://www.kaggle.com/dansbecker/using-categorical-data-with-one-hot-encoding

X_train.loc[:,1] = le.fit_transform(X_train.loc[:,1])
X_train.loc[:,2] = le.fit_transform(X_train.loc[:,2])
X_train.loc[:,3] = le.fit_transform(X_train.loc[:,3])

X_test.loc[:,1] = le.fit_transform(X_test.loc[:,1])
X_test.loc[:,2] = le.fit_transform(X_test.loc[:,2])
X_test.loc[:,3] = le.fit_transform(X_test.loc[:,3])

X_train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
0,0,1,20,9,491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,150,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00
1,0,2,44,9,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00
2,0,1,49,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00
3,0,1,24,9,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01
4,0,1,24,9,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [0]:
def build_network():
  model = Sequential()
  model.add(Dense(10, input_dim=41, kernel_initializer='uniform', activation='relu',kernel_regularizer=regularizers.l2(0.01)))
  model.add(Dense(5, kernel_initializer='uniform', activation='relu',kernel_regularizer=regularizers.l2(0.01)))
  model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid',kernel_regularizer=regularizers.l2(0.01)))
  # Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [0]:
def train_model(model, params):
  st = time.time()
  history1 = model.fit(params["x_train"], params["y_train"], validation_split=0.33, epochs=params["EPOCHS"], batch_size=10, verbose=0)
  et = time.time()
  time_taken = et - st
  return history1,time_taken

In [0]:
def run_model(params):    
    model = build_network()
    history,time_taken = train_model(model, params)
    accs = np.array(history.history["val_acc"])
    best_acc = accs.max()
    best_epoch = accs.argmax() + 1
    total_time = time_taken
    score_time = best_epoch / params["EPOCHS"]
    # print("Adjusted Score time {:3.2f}".format(total_time * score_time))
    adjustment_multiplier = 1.0 + ((params["MAX_GOAL_TIME"] - (total_time * score_time)) / params["MAX_GOAL_TIME"])

    # prevent it from only prioritizing speed
    if adjustment_multiplier > params["BEST_TIME_BONUS"]:
        adjustment_multiplier = params["BEST_TIME_BONUS"]

    fitness = best_acc * adjustment_multiplier
    return fitness, best_acc, best_epoch, total_time
    deprecation._PRINT_DEPRECATION_WARNINGS = False 

In [0]:
def run_model_x(params, x=5):
    
    f_list = []
    ba_list = []
    be_list = []
    t_list = []
    
    for idx in range(x):
        fitness, best_acc, best_epoch, total_time = run_model(params)
        
        f_list.append(fitness)
        ba_list.append(best_acc)
        be_list.append(best_epoch)
        t_list.append(total_time)
        
    median_fitness = np.median(f_list)
    
    _index = f_list.index(median_fitness)
    print("Accuracy after this mutation: ", ba_list[_index])
    print("\n")  
    return f_list[_index], ba_list[_index], be_list[_index], t_list[_index]

In [0]:
learn.lr_find()
learn.sched.plot_lr()

NameError: ignored

In [0]:
PERMUT_OFF = 3  

#img_rows, img_cols = 28, 28
    
m_params = {}

# true constants that won't change during training
m_params["MAX_GOAL_TIME"] = 45.0 
m_params["BEST_TIME_BONUS"] = 1.0 

m_params["BATCH_SIZE"] = 512      # [1 - 4096] (powers of 2)
m_params["EPOCHS"] = 10           # [10 - 50]
m_params["LEARNING_RATE"] = 0.005    # [1e1 - 1e-5]
m_params["HIDDEN_UNITS"] = 10
m_params["OPTIMIZER"] = "Adam"


def mutagen(params, previous_mutation):
    '''
    takes in a parameters and mutates something
    '''
    mutation_keys = ["LEARNING_RATE", "EPOCHS"]
    
    mutation_type = previous_mutation
    
    while mutation_type == previous_mutation:
        mutation_type = np.random.choice(mutation_keys)


    if mutation_type == "EPOCHS":

        mutations = []
      
        while len(mutations) < PERMUT_OFF:
            temp_e = np.random.randint(-10, 10)
            temp_e += params[mutation_type]

            if 2 <= temp_e <= 1000:
                if temp_e not in mutations:
                    mutations.append(temp_e)

    elif mutation_type == "BATCH_SIZE":
        mutations = [params[mutation_type]]

        while len(mutations) < PERMUT_OFF:
            temp = int(np.random.choice([256, 512, 1024, 2048, 4096, 8192, 16386]))

            if temp not in mutations:
                mutations.append(temp)

    elif mutation_type == "LEARNING_RATE":

        mutations = []

        while len(mutations) < PERMUT_OFF:
            temp = np.random.normal(scale=0.1, loc=-0.1)
            #     print(temp)

            temp = np.log10(params[mutation_type]) + temp

            if -5.0 <= temp:
                if temp >= 1.0:
                    temp = 1.0
            else:
                temp = -5.0

            lr = 10**temp

            mutations.append(lr)
    
    return mutation_type, mutations
    
    
def darwin(params, generations=500):
    log = []

    previous_mutation = None
    previous_generation_performance = 0   
    for generation in range(generations):

        mutation_type, mutations = mutagen(params, previous_mutation)
        
        print("Generation {:}: {:}".format(generation + 1, mutation_type))
        
        previous_mutation = mutation_type


        fitness_tracker = []

        time_tracker = []
        best_acc_tracker = []
        

        for mutation in mutations:
            print("Mutation Type: {:}  Mutation Value: {:}".format(mutation_type, str(mutation)))
            params[mutation_type] = mutation

            fitness, best_acc, best_epoch, total_time = run_model_x(params)
            fitness_tracker.append(fitness)
            time_tracker.append(total_time)
            best_acc_tracker.append(44best_acc)

        best_generation = np.argmax(fitness_tracker)
        best_fitness = np.max(fitness_tracker)
        print("\nThe best mutations was", best_generation,
              "with a score of", best_fitness,
              "which corresponds to a value of ", mutations[best_generation], mutation_type)
        print("\n")
        
    
        if best_fitness > previous_generation_performance:
           
            output[mutation_type] = mutations[best_generation]
            previous_generation_performance = best_fitness
            
            log.append([best_fitness,
                        mutation_type,
                        best_acc_tracker[best_generation],
                        time_tracker[best_generation]])
            
        else:
            log.append([0.0,
                        mutation_type,
                        0.0,
                        time_tracker[best_generation]])

        #burn_log(log, log_file)

        previous_mutation = mutation_type

        print(params, "\n\n")
        
        K.clear_session()
        
        print("\n\n")
    
    params["x_test"] = ""
    params["x_train"] = ""
    params["y_test"] = ""
    params["y_train"] = ""
    
    
    print("\n\n\n\n", repr(params))
    
    with open(parameter_file, "w") as outfile:
        json.dump(params, outfile, indent=4, separators=(',', ':'))
        
def burn_log(log, log_name):

    boink = pd.DataFrame(log)

    boink.columns = ["Fitness", "Mutated_Feature", "Peak_Val_Acc", "Train_Time"]

    boink.head()

    boink.to_hdf(log_name, key="log", comp_level=9)



if __name__ == "__main__":
    
    
    m_params["x_train"] = X_train
    m_params["x_test"] = X_test
    m_params["y_train"] = y_train
    m_params["y_test"] = y_test
    
    darwin(m_params, 500)
    


W0628 05:27:53.592114 140716046296960 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0628 05:27:53.642885 140716046296960 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0628 05:27:53.651425 140716046296960 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0628 05:27:53.719070 140716046296960 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0628 05:27:53.743723 140716046296960 deprecation_wrappe

Generation 1: LEARNING_RATE
Mutation Type: LEARNING_RATE  Mutation Value: 0.0035549798883724535


W0628 05:27:54.084301 140716046296960 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Accuracy after this mutation:  0.9637977426061041


Mutation Type: LEARNING_RATE  Mutation Value: 0.004874117763996316
Accuracy after this mutation:  0.9720484892064633


Mutation Type: LEARNING_RATE  Mutation Value: 0.004308868478777348


KeyboardInterrupt: ignored

In [0]:
#########################No need to see after this one#########################
#https://towardsdatascience.com/particle-swarm-optimisation-in-machine-learning-b01b1d2ad8a8
#for pso code link : https://github.com/adityaknegi/Particle-swarm-Optimization/blob/master/Particle%20swarm%20Optimization.ipynb
j = 0
while  1:
   # if(j%300==0 or j<20):
       # plot(x)
    
    count=1+count
    w=0.9 -(count*0.5)/max_count
    pos,fun_val ,pbest =map(list,zip(*x))
    pos_new =[]
    for i in range(len(v)):
        #update velocity
        v[i]=w*v[i]+C1*(np.random.rand(1)*(pbest[i]-pos[i]))+C2*(np.random.rand(1)*(gbest-pos[i]))
        pos[i]=pos[i]+v[i][0]
        if(fun(pos[i])>fun_val[i]):
            pbest[i]=pos[i]
            fun_val[i]=fun(pbest[i])
    
    x=list(zip(pos,fun_val,pbest))
    x.sort(key=lambda t : t[1])
    gbest =x[49][2]
    gbest_value=x[49][1]
    if count >max_count:
        break;
    else:
        g_pre_best=gbest
    j=j+1

In [0]:
################################## end here#################################################### No need to see after this## 

In [0]:
class SigmoidNeuron:
  #A class for sigmoid neuron
  
  def __init__(self):
    self.w = None
    self.b = None
    
  def perceptron(self, x):
    return np.dot(x, self.w.T) + self.b
  
  def sigmoid(self, x):
    return 1.0/(1.0 + np.exp(-x))
  
  def grad_w_mse(self, x, y):
    y_pred = self.sigmoid(self.perceptron(x))
    return (y_pred - y) * y_pred * (1 - y_pred) * x
  
  def grad_b_mse(self, x, y):
    y_pred = self.sigmoid(self.perceptron(x))
    return (y_pred - y) * y_pred * (1 - y_pred)
  
  def grad_w_ce(self, x, y):
    y_pred = self.sigmoid(self.perceptron(x))
    if y == 0:
      return y_pred * x
    elif y == 1:
      return -1 * (1 - y_pred) * x
    else:
      raise ValueError("y should be 0 or 1")
    
  def grad_b_ce(self, x, y):
    y_pred = self.sigmoid(self.perceptron(x))
    if y == 0:
      return y_pred 
    elif y == 1:
      return -1 * (1 - y_pred)
    else:
      raise ValueError("y should be 0 or 1")
  
  def fit(self, X, Y, epochs=1, learning_rate=1, initialise=True, loss_fn="mse", display_loss=False):
    
    # initialise w, b
    if initialise:
      self.w = np.random.randn(1, X.shape[1])
      self.b = 0
      
    if display_loss:
      loss = {}
    
    for i in tqdm_notebook(range(epochs), total=epochs, unit="epoch"):
      dw = 0
      db = 0
      for x, y in zip(X, Y):
        if loss_fn == "mse":
          dw += self.grad_w_mse(x, y)
          db += self.grad_b_mse(x, y) 
        elif loss_fn == "ce":
          dw += self.grad_w_ce(x, y)
          db += self.grad_b_ce(x, y)
          
      m = X.shape[1]    
      self.w -= learning_rate * dw/m
      self.b -= learning_rate * db/m
      
      if display_loss:
        Y_pred = self.sigmoid(self.perceptron(X))
        if loss_fn == "mse":
          loss[i] = mean_squared_error(Y, Y_pred)
        elif loss_fn == "ce":
          loss[i] = log_loss(Y, Y_pred)
    
    if display_loss:
      plt.plot(loss.values())
      plt.xlabel('Epochs')
      if loss_fn == "mse":
        plt.ylabel('Mean Squared Error')
      elif loss_fn == "ce":
        plt.ylabel('Log Loss')
      plt.show()
      
  def predict(self, X):
    Y_pred = []
    for x in X:
      y_pred = self.sigmoid(self.perceptron(x))
      Y_pred.append(y_pred)
    return np.array(Y_pred)

In [0]:
#custom cmap

my_cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["red", "yellow", "green"])



--

In [0]:
# skip this. no need to take read the file again. we already have all the data we need

# data, labels = pd.read_csv('KDDTrain+.txt',n_samples=125973, centers=4,n_features=41, random_state=0)
# print(data.shape, labels.shape)

In [0]:
# plt.scatter(data[:,0], data[:,1], c = labels, cmap = my_cmap)
# plt.show()

In [0]:
# labels_orig = labels

# #converting the multi-class to binary class
# labels = np.mod(labels_orig,2)

In [0]:
# plt.scatter(data[:,0], data[:,1], c=labels, cmap=my_cmap)
# plt.show()

In [0]:
# X_train, X_val, Y_train, Y_val = train_test_split(data, labels, stratify=labels,random_state=0)
# print(X_train.shape, X_val.shape)

In [0]:
Y_train = y_train
X_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
0,0,1,20,9,491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.00,0.00,0.00,0.00,1.00,0.00,0.00,150,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00
1,0,2,44,9,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.00,0.00,0.00,0.00,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00
2,0,1,49,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.00,1.00,0.00,0.00,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00
3,0,1,24,9,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.20,0.20,0.00,0.00,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01
4,0,1,24,9,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.00,0.00,0.00,0.00,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,0,1,49,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,121,19,0.00,0.00,1.00,1.00,0.16,0.06,0.00,255,19,0.07,0.07,0.00,0.00,0.00,0.00,1.00,1.00
6,0,1,49,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,166,9,1.00,1.00,0.00,0.00,0.05,0.06,0.00,255,9,0.04,0.05,0.00,0.00,1.00,1.00,0.00,0.00
7,0,1,49,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,117,16,1.00,1.00,0.00,0.00,0.14,0.06,0.00,255,15,0.06,0.07,0.00,0.00,1.00,1.00,0.00,0.00
8,0,1,51,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,270,23,1.00,1.00,0.00,0.00,0.09,0.05,0.00,255,23,0.09,0.05,0.00,0.00,1.00,1.00,0.00,0.00
9,0,1,49,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,133,8,1.00,1.00,0.00,0.00,0.06,0.06,0.00,255,13,0.05,0.06,0.00,0.00,1.00,1.00,0.00,0.00


In [0]:
# We need to change Y_train to an 'array' instead of a matrix

Y_train = np.array(Y_train.T)[0]

In [0]:
# test with sklearn mlp classifier
from sklearn.neural_network import MLPClassifier
classifier = MLPClassifier(hidden_layer_sizes=(20, 10, 5), max_iter=1000)

In [0]:
# fit the classifier
classifier.fit(X_train, Y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(20, 10, 5), learning_rate='constant',
              learning_rate_init=0.001, max_iter=1000, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [0]:
temp = classifier.predict(X_test)

In [0]:
corr = 0
for i in range(len(temp)):
  if temp[i] == np.array(y_test.T)[0][i]:
    corr += 1
print("Accuracy = ", corr/len(temp))

Accuracy =  0.8293559261887864


,41
0,1
1,1
2,0
3,1
4,1
5,0
6,0
7,1
8,0
9,1


## SN Classification

In [0]:
sn = SigmoidNeuron()
def load_MNIST():
    # load up data
    x_train, x_test, y_train, y_test = pd.readcsv(KDD+.txt)

    num_classes = 10

    (x_train, y_train), (x_test, y_test) = tf.keras.pd.readcsv(KDD+.txt).load_data()

    x_train = x_train.reshape(x_train.shape[0], img_rows * img_cols)
    x_test = x_test.reshape(x_test.shape[0], img_rows * img_cols)
    input_shape = (img_rows, img_cols, 1)

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

  
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    return x_train, x_test, y_train, y_test

#training the model over 2000 epochs
sn.fit(X_train.T, Y_train, epochs=2000, learning_rate=1, display_loss =True)

In [0]:
Y_pred_train = sn.predict(X_train)
Y_pred_binarized_train = (Y_pred_train >= 0.5).astype("int").ravel()
Y_pred_val = sn.predict(X_val)
Y_pred_binarized_val = (Y_pred_val >= 0.5).astype("int").ravel()

#model performance
accuracy_train = accuracy_score(Y_pred_binarized_train, Y_train)
accuracy_val = accuracy_score(Y_pred_binarized_val, Y_val)

print("Training Accuracy", round(accuracy_train, 2))
print("Validation Accuracy", round(accuracy_val,2))

In [0]:
#visualize the results

plt.scatter(X_train[:,0],X_train[:,1], c=Y_pred_binarized_train, cmap=my_cmap, s=15*(np.abs(Y_pred_binarized_train-Y_train)+.2))
plt.show()

In [0]:
class FirstFFNetwork:
  
  def __init__(self):
    self.w1 = np.random.randn()
    self.w2 = np.random.randn()
    self.w3 = np.random.randn()
    self.w4 = np.random.randn()
    self.w5 = np.random.randn()
    self.w6 = np.random.randn()
    self.b1 = 0
    self.b2 = 0
    self.b3 = 0
  
  def sigmoid(self, x):
    return 1.0/(1.0 + np.exp(-x))
  
  def forward_pass(self, x):
    self.x1, self.x2 = x
    self.a1 = self.w1*self.x1 + self.w2*self.x2 + self.b1
    self.h1 = self.sigmoid(self.a1)
    self.a2 = self.w3*self.x1 + self.w4*self.x2 + self.b2
    self.h2 = self.sigmoid(self.a2)
    self.a3 = self.w5*self.h1 + self.w6*self.h2 + self.b3
    self.h3 = self.sigmoid(self.a3)
    return self.h3
  
  def grad(self, x, y):
    self.forward_pass(x)
    
    self.dw5 = (self.h3-y) * self.h3*(1-self.h3) * self.h1
    self.dw6 = (self.h3-y) * self.h3*(1-self.h3) * self.h2
    self.db3 = (self.h3-y) * self.h3*(1-self.h3)
    
    self.dw1 = (self.h3-y) * self.h3*(1-self.h3) * self.w5 * self.h1*(1-self.h1) * self.x1
    self.dw2 = (self.h3-y) * self.h3*(1-self.h3) * self.w5 * self.h1*(1-self.h1) * self.x2
    self.db1 = (self.h3-y) * self.h3*(1-self.h3) * self.w5 * self.h1*(1-self.h1)
  
    self.dw3 = (self.h3-y) * self.h3*(1-self.h3) * self.w6 * self.h2*(1-self.h2) * self.x1
    self.dw4 = (self.h3-y) * self.h3*(1-self.h3) * self.w6 * self.h2*(1-self.h2) * self.x2
    self.db2 = (self.h3-y) * self.h3*(1-self.h3) * self.w6 * self.h2*(1-self.h2)
    
  
  def fit(self, X, Y, epochs=1, learning_rate=1, initialise=True, display_loss=False):
    
    # initialise w, b
    if initialise:
      self.w1 = np.random.randn()
      self.w2 = np.random.randn()
      self.w3 = np.random.randn()
      self.w4 = np.random.randn()
      self.w5 = np.random.randn()
      self.w6 = np.random.randn()
      self.b1 = 0
      self.b2 = 0
      self.b3 = 0
      
    if display_loss:
      loss = {}
    
    for i in tqdm_notebook(range(epochs), total=epochs, unit="epoch"):
      dw1, dw2, dw3, dw4, dw5, dw6, db1, db2, db3 = [0]*9
      for x, y in zip(X, Y):
        self.grad(x, y)
        dw1 += self.dw1
        dw2 += self.dw2
        dw3 += self.dw3
        dw4 += self.dw4
        dw5 += self.dw5
        dw6 += self.dw6
        db1 += self.db1
        db2 += self.db2
        db3 += self.db3
        
      m = X.shape[1]
      self.w1 -= learning_rate * dw1 / m
      self.w2 -= learning_rate * dw2 / m
      self.w3 -= learning_rate * dw3 / m
      self.w4 -= learning_rate * dw4 / m
      self.w5 -= learning_rate * dw5 / m
      self.w6 -= learning_rate * dw6 / m
      self.b1 -= learning_rate * db1 / m
      self.b2 -= learning_rate * db2 / m
      self.b3 -= learning_rate * db3 / m
      
      if display_loss:
        Y_pred = self.predict(X)
        loss[i] = mean_squared_error(Y_pred, Y)
    
    if display_loss:
      plt.plot(loss.values())
      plt.xlabel('Epochs')
      plt.ylabel('Mean Squared Error')
      plt.show()
      
  def predict(self, X):
    Y_pred = []
    for x in X:
      y_pred = self.forward_pass(x)
      Y_pred.append(y_pred)
    return np.array(Y_pred)

## FFNetwork Classification

In [0]:
ffn = FirstFFNetwork()
ffn.fit(X_train, Y_train, epochs = 2000, learning_rate = 0.01, display_loss = True)

NameError: ignored

In [0]:
Y_pred_train = ffn.predict(X_train)
Y_pred_binarised_train = (Y_pred_train >= 0.5).astype("int").ravel()
Y_pred_val = ffn.predict(X_val)
Y_pred_binarised_val = (Y_pred_val >= 0.5).astype("int").ravel()
accuracy_train = accuracy_score(Y_pred_binarised_train, Y_train)
accuracy_val = accuracy_score(Y_pred_binarised_val, Y_val)

print("Training accuracy", round(accuracy_train, 2))
print("Validation accuracy", round(accuracy_val, 2))

In [0]:
#visualize the results
plt.scatter(X_train[:,0], X_train[:,1], c=Y_pred_binarised_train, cmap=my_cmap, s=15*(np.abs(Y_pred_binarised_train-Y_train)+.2))
plt.show()

## Feed Forward Network - Generic Class

In [0]:
class FFSNNetwork:
  
  def __init__(self, n_inputs, hidden_sizes=[2]):
    #intialize the inputs
    self.nx = n_inputs
    self.ny = 1
    self.nh = len(hidden_sizes)
    self.sizes = [self.nx] + hidden_sizes + [self.ny]
    
    self.W = {}
    self.B = {}
    for i in range(self.nh+1):
      self.W[i+1] = np.random.randn(self.sizes[i], self.sizes[i+1])
      self.B[i+1] = np.zeros((1, self.sizes[i+1]))
  
  def sigmoid(self, x):
    return 1.0/(1.0 + np.exp(-x))
  
  def forward_pass(self, x):
    self.A = {}
    self.H = {}
    self.H[0] = x.reshape(1, -1)
    for i in range(self.nh+1):
      self.A[i+1] = np.matmul(self.H[i], self.W[i+1]) + self.B[i+1]
      self.H[i+1] = self.sigmoid(self.A[i+1])
    return self.H[self.nh+1]
  
  def grad_sigmoid(self, x):
    return x*(1-x) 
    
  def grad(self, x, y):
    self.forward_pass(x)
    self.dW = {}
    self.dB = {}
    self.dH = {}
    self.dA = {}
    L = self.nh + 1
    self.dA[L] = (self.H[L] - y)
    for k in range(L, 0, -1):
      self.dW[k] = np.matmul(self.H[k-1].T, self.dA[k])
      self.dB[k] = self.dA[k]
      self.dH[k-1] = np.matmul(self.dA[k], self.W[k].T)
      self.dA[k-1] = np.multiply(self.dH[k-1], self.grad_sigmoid(self.H[k-1]))
    
  def fit(self, X, Y, epochs=1, learning_rate=1, initialise=True, display_loss=False):
    
    # initialise w, b
    if initialise:
      for i in range(self.nh+1):
        self.W[i+1] = np.random.randn(self.sizes[i], self.sizes[i+1])
        self.B[i+1] = np.zeros((1, self.sizes[i+1]))
      
    if display_loss:
      loss = {}
    
    for e in tqdm_notebook(range(epochs), total=epochs, unit="epoch"):
      dW = {}
      dB = {}
      for i in range(self.nh+1):
        dW[i+1] = np.zeros((self.sizes[i], self.sizes[i+1]))
        dB[i+1] = np.zeros((1, self.sizes[i+1]))
      for x, y in zip(X, Y):
        self.grad(x, y)
        for i in range(self.nh+1):
          dW[i+1] += self.dW[i+1]
          dB[i+1] += self.dB[i+1]
        
      m = X.shape[1]
      for i in range(self.nh+1):
        self.W[i+1] -= learning_rate * dW[i+1] / m
        self.B[i+1] -= learning_rate * dB[i+1] / m
      
      if display_loss:
        Y_pred = self.predict(X)
        loss[e] = mean_squared_error(Y_pred, Y)
    
    if display_loss:
      plt.plot(loss.values())
      plt.xlabel('Epochs')
      plt.ylabel('Mean Squared Error')
      plt.show()
      
  def predict(self, X):
    Y_pred = []
    for x in X:
      y_pred = self.forward_pass(x)
      Y_pred.append(y_pred)
    return np.array(Y_pred).squeeze()

In [0]:
ffsnn = FFSNNetwork(2, [2, 3])
ffsnn.fit(X_train, Y_train, epochs=1000, learning_rate=.001, display_loss=True)

In [0]:
Y_pred_train = ffsnn.predict(X_train)
Y_pred_binarised_train = (Y_pred_train >= 0.5).astype("int").ravel()
Y_pred_val = ffsnn.predict(X_val)
Y_pred_binarised_val = (Y_pred_val >= 0.5).astype("int").ravel()
accuracy_train = accuracy_score(Y_pred_binarised_train, Y_train)
accuracy_val = accuracy_score(Y_pred_binarised_val, Y_val)

print("Training accuracy", round(accuracy_train, 2))
print("Validation accuracy", round(accuracy_val, 2))

In [0]:
plt.scatter(X_train[:,0], X_train[:,1], c=Y_pred_binarised_train, cmap=my_cmap, s=15*(np.abs(Y_pred_binarised_train-Y_train)+.2))
plt.show()

In [0]:
import pandas as pd

In [0]:
import numpy as np
import keras as k

## Multi class classification

In [0]:
class FFSN_MultiClass:
  
  def __init__(self, n_inputs, n_outputs, hidden_sizes=[3]):
    self.nx = n_inputs
    self.ny = n_outputs
    self.nh = len(hidden_sizes)
    self.sizes = [self.nx] + hidden_sizes + [self.ny] 

    self.W = {}
    self.B = {}
    for i in range(self.nh+1):
      self.W[i+1] = np.random.randn(self.sizes[i], self.sizes[i+1])
      self.B[i+1] = np.zeros((1, self.sizes[i+1]))
      
  def sigmoid(self, x):
    return 1.0/(1.0 + np.exp(-x))
  
  def softmax(self, x):
    exps = np.exp(x)
    return exps / np.sum(exps)

  def forward_pass(self, x):
    self.A = {}
    self.H = {}
    self.H[0] = x.reshape(1, -1)
    for i in range(self.nh):
      self.A[i+1] = np.matmul(self.H[i], self.W[i+1]) + self.B[i+1]
      self.H[i+1] = self.sigmoid(self.A[i+1])
    self.A[self.nh+1] = np.matmul(self.H[self.nh], self.W[self.nh+1]) + self.B[self.nh+1]
    self.H[self.nh+1] = self.softmax(self.A[self.nh+1])
    return self.H[self.nh+1]
  
  def predict(self, X):
    Y_pred = []
    for x in X:
      y_pred = self.forward_pass(x)
      Y_pred.append(y_pred)
    return np.array(Y_pred).squeeze()
 
  def grad_sigmoid(self, x):
    return x*(1-x) 
  
  def cross_entropy(self,label,pred):
    yl=np.multiply(pred,label)
    yl=yl[yl!=0]
    yl=-np.log(yl)
    yl=np.mean(yl)
    return yl
 
  def grad(self, x, y):
    self.forward_pass(x)
    self.dW = {}
    self.dB = {}
    self.dH = {}
    self.dA = {}
    L = self.nh + 1
    self.dA[L] = (self.H[L] - y)
    for k in range(L, 0, -1):
      self.dW[k] = np.matmul(self.H[k-1].T, self.dA[k])
      self.dB[k] = self.dA[k]
      self.dH[k-1] = np.matmul(self.dA[k], self.W[k].T)
      self.dA[k-1] = np.multiply(self.dH[k-1], self.grad_sigmoid(self.H[k-1])) 
    
  def fit(self, X, Y, epochs=100, initialize='True', learning_rate=0.01, display_loss=False):
      
    if display_loss:
      loss = {}
      
    if initialize:
      for i in range(self.nh+1):
        self.W[i+1] = np.random.randn(self.sizes[i], self.sizes[i+1])
        self.B[i+1] = np.zeros((1, self.sizes[i+1]))
        
    for epoch in tqdm_notebook(range(epochs), total=epochs, unit="epoch"):
      dW = {}
      dB = {}
      for i in range(self.nh+1):
        dW[i+1] = np.zeros((self.sizes[i], self.sizes[i+1]))
        dB[i+1] = np.zeros((1, self.sizes[i+1]))
      for x, y in zip(X, Y):
        self.grad(x, y)
        for i in range(self.nh+1):
          dW[i+1] += self.dW[i+1]
          dB[i+1] += self.dB[i+1]
                  
      m = X.shape[1]
      for i in range(self.nh+1):
        self.W[i+1] -= learning_rate * (dW[i+1]/m)
        self.B[i+1] -= learning_rate * (dB[i+1]/m)
        
      if display_loss:
        Y_pred = self.predict(X) 
        loss[epoch] = self.cross_entropy(Y, Y_pred)
    
    if display_loss:
      plt.plot(loss.values())
      plt.xlabel('Epochs')
      plt.ylabel('CE')
      plt.show()

In [0]:
X_train, X_val, Y_train, Y_val = train_test_split(data, labels_orig, stratify=labels_orig, random_state=0)
print(X_train.shape, X_val.shape, labels_orig.shape)

In [0]:
enc = OneHotEncoder()
# 0 -> (1, 0, 0, 0), 1 -> (0, 1, 0, 0), 2 -> (0, 0, 1, 0), 3 -> (0, 0, 0, 1)
y_OH_train = enc.fit_transform(np.expand_dims(Y_train,1)).toarray()
y_OH_val = enc.fit_transform(np.expand_dims(Y_val,1)).toarray()
print(y_OH_train.shape, y_OH_val.shape)

In [0]:
ffsn_multi = FFSN_MultiClass(2,4,[2,3])
ffsn_multi.fit(X_train,y_OH_train,epochs=2000,learning_rate=.005,display_loss=True)

In [0]:
Y_pred_train = ffsn_multi.predict(X_train)
Y_pred_train = np.argmax(Y_pred_train,1)

Y_pred_val = ffsn_multi.predict(X_val)
Y_pred_val = np.argmax(Y_pred_val,1)

accuracy_train = accuracy_score(Y_pred_train, Y_train)
accuracy_val = accuracy_score(Y_pred_val, Y_val)

print("Training accuracy", round(accuracy_train, 2))
print("Validation accuracy", round(accuracy_val, 2))

In [0]:
plt.scatter(X_train[:,0], X_train[:,1], c=Y_pred_train, cmap=my_cmap, s=15*(np.abs(np.sign(Y_pred_train-Y_train))+.1))
plt.show()

## Exercises

In [0]:
from sklearn.datasets import make_moons, make_circles

In [0]:
data, labels = make_moons(n_samples=1000, random_state=0, noise=0.15)
print(data.shape, labels.shape)

In [0]:
plt.scatter(data[:,0], data[:,1], c=labels, cmap=my_cmap)
plt.show()

In [0]:
X_train, X_val, Y_train, Y_val = train_test_split(data, labels, stratify=labels,random_state=0)
print(X_train.shape, X_val.shape)

In [0]:
ffsnn = FFSNNetwork(2, [2, 3, 4])
ffsnn.fit(X_train, Y_train, epochs=2000, learning_rate=.0005, display_loss=True)

In [0]:
Y_pred_train = ffsnn.predict(X_train)
Y_pred_binarised_train = (Y_pred_train >= 0.5).astype("int").ravel()
Y_pred_val = ffsnn.predict(X_val)
Y_pred_binarised_val = (Y_pred_val >= 0.5).astype("int").ravel()
accuracy_train = accuracy_score(Y_pred_binarised_train, Y_train)
accuracy_val = accuracy_score(Y_pred_binarised_val, Y_val)

print("Training accuracy", round(accuracy_train, 2))
print("Validation accuracy", round(accuracy_val, 2))

In [0]:
plt.scatter(X_train[:,0], X_train[:,1], c=Y_pred_binarised_train, cmap=my_cmap, s=15*(np.abs(Y_pred_binarised_train-Y_train)+.2))
plt.show()

## Make Circles

In [0]:
data, labels = make_circles(n_samples=1000, random_state=0, noise=0.2, factor=0.3)
print(data.shape, labels.shape)

In [0]:
plt.scatter(data[:,0], data[:,1], c=labels, cmap=my_cmap)
plt.show()

In [0]:
X_train, X_val, Y_train, Y_val = train_test_split(data, labels, stratify=labels,random_state=0)
print(X_train.shape, X_val.shape)

In [0]:
ffsnn = FFSNNetwork(2, [2, 5, 6,7])
ffsnn.fit(X_train, Y_train, epochs=1000, learning_rate=.001, display_loss=True)

In [0]:
Y_pred_train = ffsnn.predict(X_train)
Y_pred_binarised_train = (Y_pred_train >= 0.5).astype("int").ravel()
Y_pred_val = ffsnn.predict(X_val)
Y_pred_binarised_val = (Y_pred_val >= 0.5).astype("int").ravel()
accuracy_train = accuracy_score(Y_pred_binarised_train, Y_train)
accuracy_val = accuracy_score(Y_pred_binarised_val, Y_val)

print("Training accuracy", round(accuracy_train, 2))
print("Validation accuracy", round(accuracy_val, 2))

In [0]:
plt.scatter(X_train[:,0], X_train[:,1], c=Y_pred_binarised_train, cmap=my_cmap, s=15*(np.abs(Y_pred_binarised_train-Y_train)+.2))
plt.show()

In [0]:
#to be linked with this experiment too.
#same population as given in ga 
#basics - https://www.sciencedirect.com/science/article/pii/S0020025509001200



import random
import numpy
import math
from solution import solution
import time
import massCalculation
import gConstant
import gField
import move

# just need to connect a function here to ffsnn        
def GSA(objf,lb,ub,dim,PopSize,iters):
    
    ElitistCheck =1
    Rpower = 1 
     
    s=solution()
        
    """ Initializations """
    
    vel=numpy.zeros((PopSize,dim))
    fit = numpy.zeros(PopSize)
    M = numpy.zeros(PopSize)
    gBest=numpy.zeros(dim)
    gBestScore=float("inf")
    
    pos=numpy.random.uniform(0,1,(PopSize,dim)) *(ub-lb)+lb
    
    convergence_curve=numpy.zeros(iters)
    
    print("GSA is optimizing  \""+objf.__name__+"\"")    
    
    timerStart=time.time() 
    s.startTime=time.strftime("%Y-%m-%d-%H-%M-%S")
    
    for l in range(0,iters):
        for i in range(0,PopSize):
            l1 = [None] * dim
            l1=numpy.clip(pos[i,:], lb, ub)
            pos[i,:]=l1

            #Calculate objective function for each particle
            fitness=[]
            fitness=objf(l1)
            fit[i]=fitness
    
                
            if(gBestScore>fitness):
                gBestScore=fitness
                gBest=l1           
        
        """ Calculating Mass """
        M = massCalculation.massCalculation(fit,PopSize,M)

        """ Calculating Gravitational Constant """        
        G = gConstant.gConstant(l,iters)        
        
        """ Calculating Gfield """        
        acc = gField.gField(PopSize,dim,pos,M,l,iters,G,ElitistCheck,Rpower)
        
        """ Calculating Position """        
        pos, vel = move.move(PopSize,dim,pos,vel,acc)
        
        convergence_curve[l]=gBestScore
      
        if (l%1==0):
               print(['At iteration '+ str(l+1)+ ' the best fitness is '+ str(gBestScore)]);
    timerEnd=time.time()  
    s.endTime=time.strftime("%Y-%m-%d-%H-%M-%S")
    s.executionTime=timerEnd-timerStart
    s.convergence=convergence_curve
    s.Algorithm="GSA"
    s.objectivefunc=objf.__name__

    return s
         